In [1]:
import sys
sys.path.append('../')
import matplotlib.pyplot as plt
from smodels.decomposition.theorySMS import TheorySMS
from smodels.experiment.expSMS import ExpSMS
from smodels.share.models.mssm import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.experiment.defaultFinalStates import finalStates
from smodels.base.model import Model
from collections import OrderedDict
from smodels.base.physicsUnits import fb, GeV
from itertools import product
from smodels.experiment.databaseObj import Database
import copy
from smodels.experiment.expAuxiliaryFuncs import bracketToProcessStr
import re

### Get database

In [2]:
db = Database('official')

In [69]:
def convertAxesStr(inStr):
    entryDict = {}
    # First find the outermost enclosing brackets:
    iLeft = inStr.find('[')
    iRight = inStr.rfind(']')+1
    newStr = inStr[iLeft:iRight]
    # Remove blank spaces:
    newStr = newStr.replace(" ","")
    iel = 0
    # Loop over the string and replace entries by
    # "Xi" strings
    for x in re.split(r',\s*(?![^()]*\))', newStr):
        x = x.replace('[','').replace(']','')
        x = x.strip()
        if not x in entryDict:
            entryDict[x] = "'X%i'" %iel
            iel += 1
        xNew = entryDict[x]
        # print('x=',x,'xNew=',xNew)
        # print('before=',newStr)
        newStr = newStr.replace(x,xNew,1)
        # print('after=',newStr)
    # Add the original enclosing strings 
    # (for cases where the string ends in ]]*2)
    newStr = inStr[:iLeft]+newStr+inStr[iRight:]
    # print('final=',newStr)
    # Evaluate the string with the entries replaces
    # by Xi strings:
    axes = eval(newStr)
    
    return axes

def getAxesSMS(txAxes):
    """
    Generates a dummy SMS with nodes holding axis labels.
    """
    axesSMS = []
    if isinstance(txAxes,str):
        txAxes = [txAxes]
    for txAx in txAxes:
        axis = convertAxesStr(txAx)
        axisStr = str([[['anySM'] for ptc in br[:-1]] for br in axis])
        finalStates = [str(br[-1]) for br in axis]
        intermediateStates = [[str(ptc) for ptc in br[:-1]] for br in axis]
        axSMS = ExpSMS.from_string(axisStr,finalState=finalStates,
                                intermediateState=intermediateStates)
        axesSMS.append(axSMS)
    
    return axesSMS

def symmetricAxes(txAxes):

    if isinstance(txAxes,str):
        txAxes = [txAxes]
    for txAx in txAxes:
        axis = convertAxesStr(txAx)
        if len(axis) != 2:
            # print('Found %i branches!' %(len(axis)))
            return False
        if axis[0] != axis[1]:
            # print('Non-symmetric axis:\n %s != %s' %(axis[0],axis[1]))
            return False

    return True
                


In [76]:
stop = False
missingAxes = set([])
asymmetricAxes = set([])
asymmetricMultiple = set([])
for exp in db.getExpResults():
    for tx in exp.getTxNames():
        txID = exp.globalInfo.id+':'+str(tx)
        if not tx.axes:
            missingAxes.add(txID)
            continue
        sym = symmetricAxes(tx.axes)
        if not sym:
            asymmetricAxes.add(txID)
            if len(tx.smsMap) > 1:
                asymmetricMultiple.add(txID)

missingAxes = sorted(list(missingAxes))
asymmetricAxes = sorted(list(asymmetricAxes))
asymmetricMultiple = sorted(list(asymmetricMultiple))
print('Missing axes for:')
for miss in missingAxes:
    print(miss)
print('Asymmetric axes with more than 1 SMS:')
for asym in asymmetricMultiple:
    print(asym)


Missing axes for:
ATLAS-SUSY-2016-32:THSCPM5
ATLAS-SUSY-2016-32:THSCPM6
ATLAS-SUSY-2016-32:THSCPM7
CMS-EXO-13-006:THSCPM5
CMS-EXO-13-006:THSCPM6
CMS-EXO-13-006:THSCPM7
Asymmetric axes with more than 1 SMS:
ATLAS-SUSY-2019-09:TChiWZoff
